In [0]:
CATALOG_NAME = dbutils.widgets.get("catalog")
SCHEMA_NAME = dbutils.widgets.get("schema")
table = dbutils.widgets.get("table")

path = f"{catalog}.{schema}.{table}"

In [0]:
df = spark.read.table(path)
display(df)

In [0]:
import os
from urllib.parse import urlparse

urls = df.select("url").collect()

for row in urls:
    url = row['url']
    
    # Extract filename
    parsed_url = urlparse(url)
    filename = os.path.basename(parsed_url.path)
    table_name = filename.replace('.csv', '').replace('-', '_')
    
    # Download to DBFS
    dbfs_path = f"/tmp/{filename}"
    dbutils.fs.cp(url, f"dbfs:{dbfs_path}")
    
    # Load CSV
    df_csv = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load(f"dbfs:{dbfs_path}")
    
    # Drop table if it exists to avoid schema conflicts
    spark.sql(f"DROP TABLE IF EXISTS {CATALOG_NAME}.{SCHEMA_NAME}.{table_name}")
    
    # Create new table
    df_csv.write.format("delta").mode("overwrite").saveAsTable(f"{CATALOG_NAME}.{SCHEMA_NAME}.{table_name}")
    
    print(f"Created table: {table_name}")
    
    # Clean up
    dbutils.fs.rm(f"dbfs:{dbfs_path}")
